In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from pathlib import Path
import torch

In [ ]:
use_data_cache=False

model_path=Path('./models')
pretrained_mdl= "t5-small" 



In [3]:
if pretrained_mdl == "t5-small" :
    tokenizer = T5Tokenizer.from_pretrained(model_path / "t5-small-new")
    model = T5ForConditionalGeneration.from_pretrained(model_path / "t5-small-new")
    print("loading t5-small model...")

print(len(tokenizer))

loading t5-small model...
32104


In [4]:
print("Embedding matrix shape before resizing:", model.shared.weight.shape)

new_token1=["q_theta", "pow", "INT+", "INT-", "add", "mul", "z", "t"]

print(len(tokenizer))
tokenizer.add_tokens(new_token1)
print(len(tokenizer))


Embedding matrix shape before resizing: torch.Size([32104, 512])
32104
32104


In [5]:
model.resize_token_embeddings(len(tokenizer))
print(model.shared.weight.shape)
#print("Embedding matrix shape after resizing:", model.shared.weight[-len(new_token1):].shape)

torch.Size([32104, 512])


In [6]:
token_ids_to_zero = tokenizer.convert_tokens_to_ids(new_token1)
print(f"Token IDs to zero: {token_ids_to_zero}")

Token IDs to zero: [32100, 32101, 32102, 32103, 13039, 4115, 172, 17]


In [ ]:
embeddings = model.shared.weight

embed_mean = embeddings.mean().item()
embed_std = embeddings.std().item()

print(f"现有embedding统计: mean={embed_mean:.4f}, std={embed_std:.4f}")
print(f"Token IDs to initialize: {token_ids_to_zero}")

print("\nNew token embeddings after initialization:")
for token, tid in zip(new_token1, token_ids_to_zero):
    emb = embeddings[tid]
    print(f"  {token} (id={tid}): mean={emb.mean().item():.4f}, std={emb.std().item():.4f}")

现有embedding统计: mean=0.1213, std=23.1957
Token IDs to initialize: [32100, 32101, 32102, 32103, 13039, 4115, 172, 17]

New token embeddings after initialization:
  q_theta (id=32100): mean=-0.0308, std=0.4735
  pow (id=32101): mean=-0.0232, std=0.4466
  INT+ (id=32102): mean=-0.0183, std=0.4596
  INT- (id=32103): mean=-0.0049, std=0.4631
  add (id=13039): mean=-0.0067, std=0.4885
  mul (id=4115): mean=0.0220, std=0.4522
  z (id=172): mean=-0.0307, std=0.4692
  t (id=17): mean=0.0102, std=0.4832


In [8]:
model.save_pretrained("./models/t5-small-new")
tokenizer.save_pretrained("./models/t5-small-new")

('./models/t5-small-new/tokenizer_config.json',
 './models/t5-small-new/special_tokens_map.json',
 './models/t5-small-new/spiece.model',
 './models/t5-small-new/added_tokens.json')

In [9]:
tokenizer(["q_theta","mul","add","z","t","apple"], return_tensors="pt",is_split_into_words=True, padding=True, truncation=True)

{'input_ids': tensor([[32100,  4115, 13039,   172,    17,  8947,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}